# Build (EXPLOIT)

### Import libraries

In [1]:
import sqlite3
import warnings

### Ignore warnings

In [2]:
warnings.filterwarnings("ignore")

### User defined variables

In [3]:
db = "../../db.sqlite3"

### Drop table
This is for demonstration purposes

In [4]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS whale_pointsofinterest''')
conn.commit()
conn.close()

### Create `EXPLOIT` table

In [5]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS whale_pointsofinterest(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        vendor_id VARCHAR(39),
        
        sample_idx VARCHAR(40),
        area NUMERIC(4, 6),
        deviation NUMERIC(4, 6),
        epsg_code VARCHAR(6),

        status VARCHAR(20),
        locked_by INTEGER,
        review_count INTEGER,
        reviewed_by_users,

        email VARCHAR(35),
        client_ip VARCHAR(13),
        out_time DATE,
        in_time DATE,
        
        classification VARCHAR(20),
        confidence VARCHAR(10),
        species VARCHAR(50),
        comments VARCHAR(500),
        
        user1_id VARCHAR(30),
        user1_classification VARCHAR(20),
        user1_comments VARCHAR(500),
        user1_species VARCHAR(50),
        user1_confidence VARCHAR(10),

        user2_id VARCHAR(30),
        user2_classification VARCHAR(20),
        user2_comments VARCHAR(500),
        user2_species VARCHAR(50),
        user2_confidence VARCHAR(10),

        user3_id VARCHAR(30),
        user3_classification VARCHAR(20),
        user3_comments VARCHAR(500),
        user3_species VARCHAR(50),
        user3_confidence VARCHAR(10),

        final_review VARCHAR(20),
        final_review_date DATE,
        final_species VARCHAR(50),
        final_confidence VARCHAR(10),

        FOREIGN KEY (locked_by)
            REFERENCES auth_user(id)
    )
''')

c.execute('''SELECT AddGeometryColumn('whale_pointsofinterest', 'point', 4326, 'POINT')''')

conn.commit()
conn.close()

# End